In [ ]:
!pip install -q google-adk

In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass  # Not running in Colab; uses env var already set

# Fitness Coach Agent

Build an agent that helps users stay fit. Your agent will have these tools:

1. **log_workout** — Record a completed workout session
2. **calculate_calories** — Estimate calories burned for an exercise
3. **suggest_exercise** — Recommend exercises based on target muscle group and equipment

Tools return hardcoded/fake data — the goal is designing the **interface** between the model and your code.

## Setup

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.genai import types

## TODO 1: Define your tools

Just write plain Python functions. The ADK infers the schema from:
- **Type hints** → parameter types
- **Docstring** → tool description

The first tool is done for you as an example. Implement the remaining two.

In [ ]:
# Example tool (complete)

def log_workout(exercise: str, duration_minutes: int, sets: int, reps: int) -> dict:
    """Log a completed workout session.

    Args:
        exercise: Name of the exercise (e.g. "bench press", "running").
        duration_minutes: How long the workout lasted in minutes.
        sets: Number of sets completed.
        reps: Number of reps per set.

    Returns:
        dict: status and workout summary or error message.
    """
    return {
        "status": "success",
        "report": (
            f"Workout logged: {exercise} — {sets}x{reps} for {duration_minutes} min. "
            f"Great job! You've completed 3 workouts this week."
        ),
    }


def calculate_calories(exercise: str, duration_minutes: int, weight_kg: float) -> dict:
    """Estimate calories burned for a given exercise.

    Args:
        exercise: Name of the exercise (e.g. "running", "cycling", "swimming").
        duration_minutes: Duration of the exercise in minutes.
        weight_kg: User's body weight in kilograms.

    Returns:
        dict: status and calorie estimate or error message.
    """
    # TODO: implement — return a dict with "status" and "report" keys
    # Hint: use a simple formula like calories = duration * weight * factor
    pass


def suggest_exercise(muscle_group: str, equipment: str) -> dict:
    """Suggest exercises for a target muscle group and available equipment.

    Args:
        muscle_group: Target area (e.g. "chest", "legs", "back", "core").
        equipment: Available equipment (e.g. "dumbbells", "barbell", "bodyweight", "none").

    Returns:
        dict: status and exercise suggestions or error message.
    """
    # TODO: implement — return a dict with "status" and "report" keys
    # Hint: return 2-3 exercise suggestions with sets/reps recommendations
    pass

## TODO 2: Define your agent

In [ ]:
agent = Agent(
    model="gemini-3-flash-preview",
    name="fitness_coach",
    # TODO: Write an instruction that gives the agent a motivating coach persona.
    # Hint: describe its coaching style, safety reminders, encouragement approach, etc.
    instruction="""You are a helpful fitness coach.

Replace this with your agent's persona and guidelines.
""",
    tools=[log_workout, calculate_calories, suggest_exercise],
)

## Run your agent

Once you've filled in the TODOs above, run this cell to test your agent!

In [ ]:
runner = InMemoryRunner(agent=agent, app_name="fitness_coach")
runner.auto_create_session = True
user_content = types.Content(role="user", parts=[types.Part.from_text(text="I just did 30 minutes of running. Log it and tell me how many calories I burned. I weigh 75 kg.")])

final_text = ""
async for event in runner.run_async(user_id="user1", session_id="session1", new_message=user_content):
    if event.content and event.content.parts:
        for part in event.content.parts:
            if part.text:
                final_text = part.text

print(final_text)

## Tips

- **Docstrings matter** — the ADK uses them as tool descriptions. Be specific about what each parameter means.
- **Fake data is fine** — the goal is designing the agent's interface, not building a real fitness tracker.
- **Test edge cases** — what happens when a user asks about an exercise your tools don't recognize?

### Stretch Goals
- Add a `get_workout_history` tool that returns past workout logs
- Add a `create_workout_plan` tool that builds a weekly schedule
- Support different fitness goals (strength, cardio, flexibility)